In [23]:
import numpy as np
import pandas as pd
import requests
from bs4 import BeautifulSoup
from urllib.request import urlopen
from pandas.io.json import json_normalize 
import seaborn as sns
%matplotlib inline

#Getting Table from online
df_bt = pd.read_html("https://www.soccerbase.com/teams/team.sd?team_id=1310&team2_id=234&teamTabs=h2h")

#Picking Table From Source
sdf= df_bt[2]

x = str(input('Enter the starting Year in the format; 20**/20** :'))
y = x
r = x + '.1'
n = x + '.2'
m = x + '.7'
l = x + '.8'
p = x + '.9'

new_df = sdf[sdf[r].apply(lambda x: x[4])!= '/']

new_df.drop(y, axis = 1, inplace = True)

new_df.set_index(r,inplace= True)

new_df.drop([n, m,l,p], axis = 1, inplace = True)

new_df.columns = ['Home', 'Scores', 'Away', 'Result']
new_df.index.names = ['Date']


new_df['ScoresH'] = new_df['Scores'].apply(lambda x: x[0])
new_df['ScoresA'] = new_df['Scores'].apply(lambda x: x[4])

new_df['ScoresH'] = new_df['ScoresH'].apply(lambda x: int(x))
new_df['ScoresA'] = new_df['ScoresA'].apply(lambda x: int(x))
new_df['ResultN'] = new_df['ScoresH'] - new_df['ScoresA']

new_df['Result'][new_df['ResultN']>0]=new_df['Home']
new_df['Result'][new_df['ResultN']<0]=new_df['Away']
new_df['Result'][new_df['ResultN']==0]='Draw'


new_df['Result']= new_df['Result'] + ' Wins'

Result = pd.get_dummies(new_df['Result'])
Home = pd.get_dummies(new_df['Home'])
Away = pd.get_dummies(new_df['Away'])

new_df.drop(['Result','Home','Scores', 'Away'], axis = 1,inplace = True)

ddf= pd.concat([new_df,Result,Home,Away],axis = 1)

ddf.columns =['Home Goals', 'Away Goals','Goal Difference','Team A Wins','Team B Wins', 'Draw','Team A Home','Team B Home', 'Team A Away','Team B Away']


from sklearn.cluster import KMeans
kmeans = KMeans(n_clusters=2)
kmeans.fit(ddf.drop('Team A Wins',axis=1))
result = kmeans.predict(ddf.drop('Team A Wins',axis=1))


from sklearn.metrics import confusion_matrix,classification_report
print(confusion_matrix(ddf['Team A Wins'],result))
print(classification_report(ddf['Team A Wins'],result))

Enter the starting Year in the format; 20**/20** :2004/2005


C:\Users\HP\AppData\Local\Temp\ipykernel_10144\694072780.py:26: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  new_df.drop(y, axis = 1, inplace = True)
C:\Users\HP\AppData\Local\Temp\ipykernel_10144\694072780.py:30: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  new_df.drop([n, m,l,p], axis = 1, inplace = True)
C:\Users\HP\AppData\Local\Temp\ipykernel_10144\694072780.py:36: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing

[[25 18]
 [10 19]]
              precision    recall  f1-score   support

           0       0.71      0.58      0.64        43
           1       0.51      0.66      0.58        29

    accuracy                           0.61        72
   macro avg       0.61      0.62      0.61        72
weighted avg       0.63      0.61      0.61        72



In [25]:
new_df.head()

,ScoresH,ScoresA,ResultN
Date,,,
Su 10Apr 2005,0,2,-2
Tu 31Aug 2004,0,1,-1
Sa 04May 1996,2,3,-1
Sa 18Nov 1995,1,1,0
Sa 15Apr 1995,2,0,2


In [27]:
ddf.head()

,Home Goals,Away Goals,Goal Difference,Team A Wins,Team B Wins,Draw,Team A Home,Team B Home,Team A Away,Team B Away
Date,,,,,,,,,,
Su 10Apr 2005,0,2,-2,0,0,1,1,0,0,1
Tu 31Aug 2004,0,1,-1,1,0,0,0,1,1,0
Sa 04May 1996,2,3,-1,1,0,0,0,1,1,0
Sa 18Nov 1995,1,1,0,0,1,0,1,0,0,1
Sa 15Apr 1995,2,0,2,0,0,1,0,1,1,0


In [26]:
result

array([0, 0, 0, 0, 1, 1, 0, 1, 0, 0, 0, 1, 1, 0, 0, 0, 0, 1, 0, 1, 1, 1,
       1, 0, 1, 1, 0, 0, 1, 0, 1, 0, 1, 1, 1, 0, 1, 1, 1, 1, 1, 1, 0, 0,
       1, 1, 0, 0, 0, 0, 1, 1, 0, 0, 1, 0, 0, 1, 1, 0, 1, 0, 1, 1, 0, 1,
       1, 0, 0, 1, 0, 1])